In [1]:
import xarray as xr
import pandas as pd
import numpy as np

### Geting data format from PyPSA'ssp data set

In [2]:
#load hourly data and format from PyPSA-Earth's ssp data base

xr_data_path = r'C:\Users\Carlos\OneDrive - Universite de Liege\Documents\2024\Articulos y publicaciones\ECEMP\Harmonization\Demand\SouthAmerica.nc'
xr_data = xr.open_dataarray(xr_data_path)
xr_data

<xarray.DataArray 'Electricity demand' (region_code: 14, time: 8760)>
[122640 values with dtype=float64]
Coordinates:
  * region_code  (region_code) <U2 'AR' 'BO' 'BR' 'CL' ... 'PY' 'SR' 'UY' 'VE'
  * time         (time) datetime64[ns] 2013-01-01 ... 2013-12-31T23:00:00
    region_name  (region_code) <U16 ...

In [12]:
#transform nc data into dataframe
df = xr_data.to_dataframe()
df

region_name  Electricity demand
region_code time                                               
AR          2013-01-01 00:00:00   Argentina        25204.670505
            2013-01-01 01:00:00   Argentina        24914.320557
            2013-01-01 02:00:00   Argentina        24265.144747
            2013-01-01 03:00:00   Argentina        22331.379630
            2013-01-01 04:00:00   Argentina        21001.189997
...                                     ...                 ...
VE          2013-12-31 19:00:00   Venezuela        15753.699921
            2013-12-31 20:00:00   Venezuela        15753.699921
            2013-12-31 21:00:00   Venezuela        15835.144409
            2013-12-31 22:00:00   Venezuela        16119.812367
            2013-12-31 23:00:00   Venezuela        16762.794854

[122640 rows x 2 columns]

In [3]:
#check country specific data
region_data = xr_data.where((xr_data['region_code'] == 'BO') & (xr_data['region_name'] == 'Bolivia'), drop=True)
region_data

<xarray.DataArray 'Electricity demand' (region_code: 1, time: 8760)>
array([[1342.44 , 1530.55 , 1510.34 , ..., 1310.011, 1289.809, 1259.842]])
Coordinates:
  * region_code  (region_code) <U2 'BO'
  * time         (time) datetime64[ns] 2013-01-01 ... 2013-12-31T23:00:00
    region_name  (region_code) <U16 'Bolivia'

### get data from historical (custom?) sources

In [6]:
#load historic data for Bolivia in 2050 and change them from GW to MW
historic_data_path = r"C:\Users\Carlos\OneDrive - Universite de Liege\Documents\2024\Articulos y publicaciones\ECEMP\Harmonization\Demand\Electricity Generation_Hourly_v3.xlsx"
historic_data = pd.read_excel(historic_data_path, header = 0)
historic_data["demand"] = historic_data["demand"]*1000
historic_data

,hour,demand
0,1,3498.952364
1,2,3055.415413
2,3,2718.640941
3,4,2459.661119
4,5,2343.959973
...,...,...
8755,8756,7069.712498
8756,8757,6826.849639
8757,8758,6205.796594
8758,8759,4906.767009


In [7]:
#format the historic dataframe to match the base dataframe
historic_data = historic_data.rename(columns={'hour':'time','demand':'Electricity demand'})
#historic_data = historic_data.drop(columns=['%_elec (Normalized value)'])
#historic_data = historic_data.drop(index=[8760,8761])
historic_data

,time,Electricity demand
0,1,3498.952364
1,2,3055.415413
2,3,2718.640941
3,4,2459.661119
4,5,2343.959973
...,...,...
8755,8756,7069.712498
8756,8757,6826.849639
8757,8758,6205.796594
8758,8759,4906.767009


In [8]:
#change the hour format in the column time (be sure that hourly data is in CET format/time)
historic_data['time'] = pd.to_datetime('2013-01-01') + pd.to_timedelta(historic_data['time'] - 1, unit='h')
historic_data

,time,Electricity demand
0,2013-01-01 00:00:00,3498.952364
1,2013-01-01 01:00:00,3055.415413
2,2013-01-01 02:00:00,2718.640941
3,2013-01-01 03:00:00,2459.661119
4,2013-01-01 04:00:00,2343.959973
...,...,...
8755,2013-12-31 19:00:00,7069.712498
8756,2013-12-31 20:00:00,6826.849639
8757,2013-12-31 21:00:00,6205.796594
8758,2013-12-31 22:00:00,4906.767009


In [9]:
#add the missing columns/coordinates 
historic_data['region_code'] = 'BO'
historic_data['region_name'] = 'Bolivia'
historic_data = historic_data[['region_code','time','region_name','Electricity demand']]
historic_data

,region_code,time,region_name,Electricity demand
0,BO,2013-01-01 00:00:00,Bolivia,3498.952364
1,BO,2013-01-01 01:00:00,Bolivia,3055.415413
2,BO,2013-01-01 02:00:00,Bolivia,2718.640941
3,BO,2013-01-01 03:00:00,Bolivia,2459.661119
4,BO,2013-01-01 04:00:00,Bolivia,2343.959973
...,...,...,...,...
8755,BO,2013-12-31 19:00:00,Bolivia,7069.712498
8756,BO,2013-12-31 20:00:00,Bolivia,6826.849639
8757,BO,2013-12-31 21:00:00,Bolivia,6205.796594
8758,BO,2013-12-31 22:00:00,Bolivia,4906.767009


### checking if data allocation in dataframe makes sense

In [10]:
multiindex = historic_data.set_index(['region_code','time'])
multiindex

region_name  Electricity demand
region_code time                                               
BO          2013-01-01 00:00:00     Bolivia         3498.952364
            2013-01-01 01:00:00     Bolivia         3055.415413
            2013-01-01 02:00:00     Bolivia         2718.640941
            2013-01-01 03:00:00     Bolivia         2459.661119
            2013-01-01 04:00:00     Bolivia         2343.959973
...                                     ...                 ...
            2013-12-31 19:00:00     Bolivia         7069.712498
            2013-12-31 20:00:00     Bolivia         6826.849639
            2013-12-31 21:00:00     Bolivia         6205.796594
            2013-12-31 22:00:00     Bolivia         4906.767009
            2013-12-31 23:00:00     Bolivia         3966.127453

[8760 rows x 2 columns]

In [13]:
selected_range_df = df.loc[('BO', '2013-01-01 00:00:00'):('BO', '2013-12-31 23:00:00')]
selected_range_df

region_name  Electricity demand
region_code time                                               
BO          2013-01-01 00:00:00     Bolivia            1342.440
            2013-01-01 01:00:00     Bolivia            1530.550
            2013-01-01 02:00:00     Bolivia            1510.340
            2013-01-01 03:00:00     Bolivia            1456.860
            2013-01-01 04:00:00     Bolivia            1366.940
...                                     ...                 ...
            2013-12-31 19:00:00     Bolivia            1321.310
            2013-12-31 20:00:00     Bolivia            1337.062
            2013-12-31 21:00:00     Bolivia            1310.011
            2013-12-31 22:00:00     Bolivia            1289.809
            2013-12-31 23:00:00     Bolivia            1259.842

[8760 rows x 2 columns]

In [14]:
#replace historic data into the data frame 
modified_df = df
modified_df.loc[('BO','2013-01-01 00:00:00'):('BO','2013-12-31 23:00:00'), 'Electricity demand'] = multiindex['Electricity demand']
modified_df

region_name  Electricity demand
region_code time                                               
AR          2013-01-01 00:00:00   Argentina        25204.670505
            2013-01-01 01:00:00   Argentina        24914.320557
            2013-01-01 02:00:00   Argentina        24265.144747
            2013-01-01 03:00:00   Argentina        22331.379630
            2013-01-01 04:00:00   Argentina        21001.189997
...                                     ...                 ...
VE          2013-12-31 19:00:00   Venezuela        15753.699921
            2013-12-31 20:00:00   Venezuela        15753.699921
            2013-12-31 21:00:00   Venezuela        15835.144409
            2013-12-31 22:00:00   Venezuela        16119.812367
            2013-12-31 23:00:00   Venezuela        16762.794854

[122640 rows x 2 columns]

In [15]:
selected_range = modified_df.loc[('BO', '2013-01-01 00:00:00'):('BO', '2013-12-31 23:00:00')]
selected_range

region_name  Electricity demand
region_code time                                               
BO          2013-01-01 00:00:00     Bolivia         3498.952364
            2013-01-01 01:00:00     Bolivia         3055.415413
            2013-01-01 02:00:00     Bolivia         2718.640941
            2013-01-01 03:00:00     Bolivia         2459.661119
            2013-01-01 04:00:00     Bolivia         2343.959973
...                                     ...                 ...
            2013-12-31 19:00:00     Bolivia         7069.712498
            2013-12-31 20:00:00     Bolivia         6826.849639
            2013-12-31 21:00:00     Bolivia         6205.796594
            2013-12-31 22:00:00     Bolivia         4906.767009
            2013-12-31 23:00:00     Bolivia         3966.127453

[8760 rows x 2 columns]

In [16]:
#defining regions
index_df = modified_df.reset_index()
index_df
regions = {c:n for c, n in zip(index_df.region_code, index_df.region_name)}
regions

{'AR': 'Argentina',
 'BO': 'Bolivia',
 'BR': 'Brazil',
 'CL': 'Chile',
 'CO': 'Colombia',
 'EC': 'Ecuador',
 'FK': 'Falkland Islands',
 'GF': 'French Guiana',
 'GY': 'Guyana',
 'PE': 'Peru',
 'PY': 'Paraguay',
 'SR': 'Suriname',
 'UY': 'Uruguay',
 'VE': 'Venezuela'}

In [17]:
#transform the modified df into nc file
xr_modified_df = modified_df.to_xarray()
xr_modified_df

<xarray.Dataset>
Dimensions:             (region_code: 14, time: 8760)
Coordinates:
  * region_code         (region_code) object 'AR' 'BO' 'BR' ... 'SR' 'UY' 'VE'
  * time                (time) datetime64[ns] 2013-01-01 ... 2013-12-31T23:00:00
Data variables:
    region_name         (region_code, time) object 'Argentina' ... 'Venezuela'
    Electricity demand  (region_code, time) float64 2.52e+04 ... 1.676e+04

In [19]:
#make the region_name variable into a coordinate
xr_modified_df = xr_modified_df.assign_coords(
    {"region_name":("region_code", [name for (code, name) in regions.items()])}
)
xr_modified_df

<xarray.Dataset>
Dimensions:             (region_code: 14, time: 8760)
Coordinates:
  * region_code         (region_code) object 'AR' 'BO' 'BR' ... 'SR' 'UY' 'VE'
  * time                (time) datetime64[ns] 2013-01-01 ... 2013-12-31T23:00:00
    region_name         (region_code) <U16 'Argentina' 'Bolivia' ... 'Venezuela'
Data variables:
    Electricity demand  (region_code, time) float64 2.52e+04 ... 1.676e+04

In [20]:
xr_modified_df.to_netcdf(r"C:\Users\Carlos\OneDrive - Universite de Liege\Documents\2024\Articulos y publicaciones\ECEMP\Harmonization\Demand\newprofile2050\era5_2013\SouthAmerica.nc")